# Conversation

**User**

I need to write a Python script that fetches cryptocurrency market data for the top 5 currencies in the last 24 months and plot the price trends.


**Assistant**

We may use `ccxt` to fetch cryptocurrency market data and `matplotlib` to visualize it. Here's a step-by-step guide:

1. **Install Required Libraries**: Make sure you have `ccxt` and `matplotlib` installed in your environment. You can install them using pip:

   ```shell
   pip install ccxt matplotlib
   ```

1. **Fetch Cryptocurrency Data**: Use the `ccxt` library to connect to a cryptocurrency exchange (e.g., Binance) and fetch the historical data for the top 5 cryptocurrencies by market capitalization. The top 5 can vary, but typically includes Bitcoin (BTC), Ethereum (ETH), and others.

1. **Data Preparation**: Prepare the data for visualization. This involves selecting the relevant time frame (last 24 months) and data points (e.g., closing prices).

1. **Visualization with Matplotlib**: Use `matplotlib` to visualize the data. You can plot the price trends for these currencies over time.

Here is a basic script that outlines how you might approach this task. Note that due to the dynamic nature of cryptocurrency rankings, you'll need to manually specify the top 5 currencies or find another way to dynamically fetch the top currencies' symbols.

First, let's write a function the returns a DataFrame of the historical data for a given currency via Binance.

In [ ]:
pip install ccxt matplotlib

In [ ]:
# Assistant

import ccxt
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Initialize exchange
exchange = ccxt.binance()


def fetch_data(symbol, timeframe='1d', since=None, limit=100):
    """
    Fetches historical OHLCV data for a given cryptocurrency symbol from Binance.

    Parameters:
    - symbol (str): The symbol of the cryptocurrency to fetch data for
    (e.g., 'BTC/USDT').
    - timeframe (str, optional): The timeframe for the data
    ('1m', '1h', '1d', etc.). Default is '1d'.
    - since (str, optional): The start date for the data in ISO8601 format.
    If not provided, defaults to 'limit' days ago.
    - limit (int, optional): The maximum number of data points to fetch.
    Default is 100.

    Returns:
    - pandas.DataFrame: A DataFrame containing the OHLCV data with timestamps
    converted to datetime objects.
    """
    # Calculate 'since' in milliseconds
    if since:
        since = exchange.parse8601(since)
    else:
        # Default to 'limit' days ago
        since = exchange.milliseconds() - 86400000 * limit

    # Fetch the OHLCV data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)

    # Convert to DataFrame
    df = pd.DataFrame(
        ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    return df

**Assistant**

Now, let's plot the data for 5 selected cryptocurrencies of interest:

In [ ]:
# Assistant

# Symbols for top 5 cryptocurrencies, you might need to update this list
symbols = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'ADA/USDT', 'XRP/USDT']

# Fetch and plot data for the top 5 cryptocurrencies
plt.figure(figsize=(14, 7))

for symbol in symbols:
    # Fetch data for the past 24 months, adjust the limit accordingly
    data = fetch_data(symbol, since=(
        datetime.now() - timedelta(days=24*30)).isoformat(), limit=720)
    plt.plot(data['timestamp'], data['close'], label=symbol)

plt.title('Cryptocurrency Prices Over the Last 24 Months')
plt.xlabel('Date')
plt.ylabel('Price in USDT')
plt.legend()
plt.show()

**Assistant**

### Key Points:
- **Symbol Format**: Cryptocurrency symbols are usually formatted as "BASE/QUOTE", e.g., "BTC/USDT". Ensure you use the correct format for the exchange you're querying.
- **Timeframe and Limit**: Adjust the `timeframe` and `limit` parameters based on your data needs. The `timeframe` can be '1d' for daily prices, and `limit` depends on how many data points you want to fetch. The limit is set high here to attempt fetching data for the whole 24 months, but be aware that different exchanges have different limitations on how much historical data can be fetched in one call.
- **Data Availability**: Not all cryptocurrencies might have 24 months of data available, especially newer ones.
- **API Rate Limits**: Be mindful of the exchange's API rate limits to avoid getting temporarily banned for making too many requests.
- **Restricted Locations**: Per Binance's Terms of Use, the access is restricted in some countries [Binance website](https://www.binance.com/en/legal/list-of-prohibited-countries).

**User**

Change the plot to display the relative price fluctuation instead of absolute prices, to make it easy to compare between the currencies.


**Assistant**

To compare the relative price fluctuation of the top 5 cryptocurrencies instead of their absolute prices, we can modify the script to plot the percentage change of each currency relative to its starting price in the period considered. This way, we can easily compare the performance of each currency regardless of their price scales.

Here's how you can adjust the script:

1. Calculate the percentage change for each currency relative to the first data point's closing price.
2. Plot these percentage changes instead of the absolute prices.

Let's modify the previous script to implement these changes:

In [ ]:
# Assistant

# .. no changes needed to the imports, initialization, and fetch_data method

# Fetch and plot data for the top 5 cryptocurrencies
plt.figure(figsize=(14, 7))

for symbol in symbols:
    # Fetch data for the past 24 months, adjust the limit accordingly
    data = fetch_data(symbol, since=(
        datetime.now() - timedelta(days=24*30)).isoformat(), limit=720)

    # Calculate the percentage change relative to the first data point
    data['percentage_change'] = (
        data['close'] - data['close'].iloc[0]) / data['close'].iloc[0] * 100

    plt.plot(data['timestamp'], data['percentage_change'], label=symbol)

plt.title('Relative Cryptocurrency Price Fluctuations Over the Last 24 Months')
plt.xlabel('Date')
plt.ylabel('Percentage Change')
plt.legend()
plt.show()

**Assistant**

### Modifications Explained:
- **Percentage Change Calculation**: This is done by subtracting the initial closing price from the current closing price, dividing by the initial closing price, and then multiplying by 100 to get a percentage.
- **Plotting Percentage Change**: The `plt.plot` calls now use `data['percentage_change']` to plot the relative price fluctuation over time.

This approach allows you to visually compare the performance of different cryptocurrencies on the same scale, highlighting their volatility and relative gains or losses over the last 24 months.

**User**

Please, change the labels to the currencies codes and also their names.

**Assistant**

To include both the code and name of the currencies in the labels for the plot, you'll need to map the currency codes to their full names. Let's update the script to include a dictionary that maps these codes to names, and then modify the label in the plot accordingly.

Note that the full names of the currencies can be quite long, so it might be helpful to use abbreviations or shorter forms if necessary to keep the plot legible. Here's how you could do it:

In [ ]:
# Assistant

# ... no changes needed to the imports, initialization, and fetch_data method

# Currency symbols and their corresponding names
symbols_and_names = {
    'BTC/USDT': 'Bitcoin (BTC)',
    'ETH/USDT': 'Ethereum (ETH)',
    'BNB/USDT': 'Binance Coin (BNB)',
    'ADA/USDT': 'Cardano (ADA)',
    'XRP/USDT': 'Ripple (XRP)'
}

# Fetch and plot data for the top 5 cryptocurrencies
plt.figure(figsize=(14, 7))

for symbol, name in symbols_and_names.items():
    # Fetch data for the past 24 months, adjust the limit accordingly
    data = fetch_data(symbol, since=(
        datetime.now() - timedelta(days=24*30)).isoformat(), limit=720)

    # Calculate the percentage change relative to the first data point
    data['percentage_change'] = (
        data['close'] - data['close'].iloc[0]) / data['close'].iloc[0] * 100

    # Update label to include both code and name
    plt.plot(data['timestamp'], data['percentage_change'], label=f"{name}")

plt.title('Relative Cryptocurrency Price Fluctuations Over the Last 24 Months')
plt.xlabel('Date')
plt.ylabel('Percentage Change')
plt.legend()
plt.show()

**Assistant**

### Key Changes:
- **`symbols_and_names` Dictionary**: This maps the cryptocurrency symbols (e.g., 'BTC/USDT') to a string that includes both the code and the name of the currency (e.g., 'Bitcoin (BTC)').
- **Label Update in Plotting**: The `label` parameter in the `plt.plot` call now uses `f"{name}"` to include both the code and the full name of each cryptocurrency in the legend.

This adjustment makes the plot more informative by clearly indicating which currency each line represents, not just by its trading symbol but also by its common name.

**User**

May I use ccxt to infer the top 5 cryptocurrencies?

**Assistant**

To infer the top 5 cryptocurrencies using `ccxt`, you would typically fetch trading volume or price data from a specific exchange and use this as a proxy for popularity or market capitalization. Keep in mind, this method might not perfectly align with global market capitalization rankings but can give you an idea of the most traded or valuable cryptocurrencies on that particular exchange.

Here's a conceptual approach to identify popular cryptocurrencies based on trading volume from an exchange:

1. **Fetch tickers for all markets**: Use `fetch_tickers()` method to get the latest ticker information for all symbols. This method provides the `quoteVolume` for the last 24 hours, which you can use to gauge activity.
2. **Sort markets by `quoteVolume`**: After fetching the tickers, sort them by `quoteVolume` to find the most traded markets.
3. **Select top markets**: From the sorted list, select the top 5 markets based on their trading volume.

Here's an example of how to implement it:

In [ ]:
# Assistant

import ccxt


# Initialize exchange
exchange = ccxt.binance()


# Ensure the exchange has the capability
if exchange.has['fetchTickers']:
    # Fetch tickers for all symbols
    tickers = exchange.fetch_tickers()
    # Extract and sort tickers by quoteVolume, descending
    sorted_tickers = sorted(tickers.items(
    ), key=lambda x: x[1]['quoteVolume'] if 'quoteVolume' in x[1] else 0, reverse=True)

    # Select top 5
    top_5_tickers = sorted_tickers[:5]

    print("Top 5 Cryptocurrencies by Trading Volume on Binance:")
    for symbol, ticker in top_5_tickers:
        print(f"{symbol} - Volume: {ticker['quoteVolume']}")
else:
    print("This exchange does not support fetching all tickers at once.")

**Assistant**

### Important Considerations:

- **Exchange Support**: Not all exchanges support fetching all tickers at once. You should check if the exchange you're interested in supports this feature (`exchange.has['fetchTickers']`).
- **API Rate Limits**: Fetching a lot of data at once can be intensive on the API and may lead to hitting rate limits. Always be mindful of the exchange's API usage policies.
- **Market Selection**: This method considers the trading volume as the main criterion for popularity. It's a common approach but doesn't account for other factors like market capitalization or liquidity across multiple exchanges.

Remember, the `quoteVolume` represents the trading volume in the quote currency over the last 24 hours, which is a useful metric for assessing how much of a particular currency has been traded in that period.